# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
#Загрузка основных библиотек
import pandas as pd
import numpy as np


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
# загрузка дадасетов
try:
    geo_0 = pd.read_csv("/datasets/geo_data_0.csv")
    geo_1 = pd.read_csv("/datasets/geo_data_1.csv")
    geo_2 = pd.read_csv("/datasets/geo_data_2.csv")
except:
    geo_0 = pd.read_csv("geo_data_0.csv")
    geo_1 = pd.read_csv("geo_data_1.csv")
    geo_2 = pd.read_csv("geo_data_2.csv")

In [3]:
def base_analizez(data):
    ''' функция для изучения датасета'''
    
    print('Просмотр датасета:')
    display(data.head())
    
    print('\n')
    print('Общая информация об датасете:')
    print(data.info())
    
    print('\n')
    print('Проверка на наличие значений nan в датасете:')
    display(pd.DataFrame(round(
        data.isna().mean()*100,)).style.background_gradient('coolwarm'))
    
    print('\n')
    print('Изучение корреляции между данными')
    print(data.corr())

In [4]:
# просмотр значений датасета
base_analizez(geo_0)

Просмотр датасета:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647




Общая информация об датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Проверка на наличие значений nan в датасете:


,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000




Изучение корреляции между данными
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


In [5]:
# просмотр значений датасета
base_analizez(geo_1)

Просмотр датасета:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305




Общая информация об датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Проверка на наличие значений nan в датасете:


,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000




Изучение корреляции между данными
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


In [6]:
# просмотр значений датасета
base_analizez(geo_2)

Просмотр датасета:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746




Общая информация об датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Проверка на наличие значений nan в датасете:


,0
id,0.000000
f0,0.000000
f1,0.000000
f2,0.000000
product,0.000000




Изучение корреляции между данными
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


**Вывод:** Во всех датасетах 100 тыс. строчек. Пропуски отсутствуют. В датасете geo_1 наблюдается полная корреляция между колонками f2 и product. 

## Обучение и проверка модели

In [7]:
def split_data(data):
    """ деление датасета на тестовую и
    валидационную выборки"""
    features = data.drop(['id', 'product'], axis = 1) 
    target =  data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25,  random_state=12345)
    
    return features, target, features_train, \
        features_valid, target_train, target_valid
    

In [8]:
def create_model_LinearRegression(features, target,
                                  features_train, features_valid,
                                 target_train, target_valid):
    """ Функция создает модель линейной регресии из входных данных
    (датасет признаков и целевого признака, тренировочных и валидационных
    выборок)"""
    
    # обучение модели
    model = LinearRegression()
    model.fit(features_train, target_train)

    # расчёт метрик по валидационной выборке
    predicted = model.predict(features_valid)
    rmse = (mean_squared_error(target_valid, predicted)) ** 0.5

    print('Cреднее предсказанное значение по валидационной выборке=', predicted.mean())
    print('RMSE =', rmse)
    
    # оценка модели  через кросс-валидацию
    scores = cross_val_score(model, features, target, cv= 5)
    final_score = scores.mean()
    print()
    print('Средняя оценка качества модели :', final_score)
    
    
    return predicted
    
    

In [9]:
# модель регрессии для датасета geo_0

# переменные для модели 0
features_geo_0, target_geo_0, features_train, features_valid_geo_0, \
target_train, target_valid_geo_0 = split_data(geo_0)

# сохранение предсказазаний модели
predicted_geo_0 = create_model_LinearRegression(features_geo_0, target_geo_0,
                                                features_train, features_valid_geo_0,
                                                target_train, target_valid_geo_0)

Cреднее предсказанное значение по валидационной выборке= 92.59256778438035
RMSE = 37.5794217150813

Средняя оценка качества модели : 0.27549130726904475


In [10]:
# модель регрессии для датасета geo_1

# переменные для модели 1
features_geo_1, target_geo_1, features_train, features_valid_geo_1,\
target_train, target_valid_geo_1 = split_data(geo_1)

# сохранение предсказазаний модели
predicted_geo_1 = create_model_LinearRegression(features_geo_1, target_geo_1,
                                                features_train, features_valid_geo_1,
                                                target_train, target_valid_geo_1)


Cреднее предсказанное значение по валидационной выборке= 68.728546895446
RMSE = 0.893099286775617

Средняя оценка качества модели : 0.9996243728923553


In [11]:
# модель регрессии для датасета geo_2

# переменные для модели 2
features_geo_2, target_geo_2, features_train, features_valid_geo_2,\
target_train, target_valid_geo_2 = split_data(geo_2)

# сохранение предсказазаний модели
predicted_geo_2 = create_model_LinearRegression(features_geo_2, target_geo_2,
                                                features_train, features_valid_geo_2,
                                                target_train, target_valid_geo_2)


Cреднее предсказанное значение по валидационной выборке= 94.96504596800489
RMSE = 40.02970873393434

Средняя оценка качества модели : 0.1987156246205129


**Вывод:** Для каждого датасета была создана отдельная модель. Самая точна модель у датасета geo_0. Самая большая ошибка по RMSE -  у модели geo_2. У данной модели также наблюдается низкое качество при кросс-валидации. В модели model_geo_1 присутствует 100% коррелирующая переменная из-за чего точность модели практически 100%.

## Подготовка к расчёту прибыли

In [12]:
# константы
COAST_START = 10_000_000_000 # бюджет на разработку
PRICE_ONE_UNIT = 450_000 # стоимость 1 тыс. барелей запасов

In [13]:
#Минимальное необходимое тыс. баррелей нефти для выходжа в 0 по проекту
min_unit_needed = round(COAST_START / PRICE_ONE_UNIT, 2) 
print(f"Минимально необходимо разработать {min_unit_needed:,} тыс. баррелей")

Минимально необходимо разработать 22,222.22 тыс. баррелей


По условию задачи для разработки выбираются 200 точек

In [14]:
# среднее необходимое кол-во запасов в каждой точке 
mean_unit_needed = round(min_unit_needed / 200, 2)
mean_unit_needed

111.11

In [15]:
def average(text:str, predicted):
    text = f"Разница между средним предсказанным значением и \
необходимым средним у модели {text} ="
    print(text, round(predicted.mean() - mean_unit_needed, 2))

In [16]:
average('geo_0', predicted_geo_0)
average('geo_1', predicted_geo_1)
average('geo_2', predicted_geo_2)

Разница между средним предсказанным значением и необходимым средним у модели geo_0 = -18.52
Разница между средним предсказанным значением и необходимым средним у модели geo_1 = -42.38
Разница между средним предсказанным значением и необходимым средним у модели geo_2 = -16.14


In [17]:
def profit(target, predict, count):
    """Функция получает на вход target, предсказанный объем запасов,
    и размер диапазона для суммирования. Из суммы вычитается константа
    расходов на установку скважин"""
    target = target.reset_index(drop = True)
    
    probs_sorted = pd.Series(predict).sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return PRICE_ONE_UNIT * selected.sum() 

In [18]:
total_price_geo_0 = profit(target_valid_geo_0, predicted_geo_0, 200)
total_price_geo_1 = profit(target_valid_geo_1, predicted_geo_1, 200)
total_price_geo_2 = profit(target_valid_geo_2, predicted_geo_2, 200)

print(f"Суммарная стоимость 200 лучших точек для geo_0: {total_price_geo_0:,}")
print(f'Суммарная стоимость 200 лучших точек для geo_1: {total_price_geo_1:,}')
print(f'Суммарная стоимость 200 лучших точек для geo_2: {total_price_geo_2:,}')

Суммарная стоимость 200 лучших точек для geo_0: 13,320,826,043.13985
Суммарная стоимость 200 лучших точек для geo_1: 12,415,086,696.68151
Суммарная стоимость 200 лучших точек для geo_2: 12,710,349,963.599833


**Вывод:** У 200 самых крупных точек в каждом датасете значение запасов переходит минимальный оцененный порог необхожимый для окупаемости проекта. Но среднее значение запасов из датасетов ниже порогового.

## Расчёт прибыли и рисков 

In [19]:
def bootstrap(target, predict, n, top_n):
    '''функция для проведения проверки bootstrap 1000 раз'''
    
    state = np.random.RandomState(12345)
    
    # сброс индексов
    target = target.reset_index(drop = True)
    values = []
    
    #создание выборки   
    for i in range(1000):
        target_sub = target.sample(n = n, replace = True, random_state = state)
        predicted = predict[target_sub.index]
        values.append(profit(target_sub, predicted, top_n) - COAST_START)

    values = pd.Series(values)
    
    #границы доверительного интервала
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    #параметры получившейся выборки  
    mean = values.mean()
    
    count = sum([1 for x in values if x<=0]) #кол-во отрицательных значений
    risk = count / len(values) 
    
    
    print(f"Средняя выручка: {mean:,.2f}")
    print(f"97,5%-квантиль: {upper:,.2f}")
    print(f"2.5%-квантиль: {lower:,.2f}")
    print(f"Риск убытков: {risk:,.2%}")


При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Поэтому будут случайным образом выбираться 500 точек, из них сохраняться 200 с лучшим результатом.

In [20]:
# bootstrap для модели geo_0
bootstrap(target_valid_geo_0, predicted_geo_0, 500, 200)

Средняя выручка: 396,164,984.80
97,5%-квантиль: 909,766,941.55
2.5%-квантиль: -111,215,545.89
Риск убытков: 6.90%


In [21]:
# bootstrap для модели geo_1
bootstrap(target_valid_geo_1, predicted_geo_1, 500, 200)

Средняя выручка: 456,045,105.79
97,5%-квантиль: 852,289,453.87
2.5%-квантиль: 33,820,509.40
Риск убытков: 1.50%


In [22]:
# bootstrap для модели geo_2
bootstrap(target_valid_geo_2, predicted_geo_2, 500, 200)

Средняя выручка: 404,403,866.57
97,5%-квантиль: 950,359,574.92
2.5%-квантиль: -163,350,413.40
Риск убытков: 7.60%


**Вывод:** Среднее распределение значений показало, что у всех точек присутствует большой риск получения убытков, но средняя выручка во всех датасетах имеет положительное значение. Наилучший результат показала модель geo_1.

<div style="border:solid steelgreen 4px; padding: 20px; border-radius: 15px">
<h1>Итог:</h1>

>В ходе работы были выполнены следующие операции:
>>- Изучена общая информация о представленном датасетах
>>- Исследованы результаты работы модели линейной регресии
>>- Оценка моделей через кросс-валидационные выборки   
>>- Исследована предсказанное значение объема месторождений
>>- Техникой Boostsrap оценены риски разработки месторождений 
   

> В ходе исследования были изучены разные датасеты для нескольких регионах. При помощи линейной регрессии и техники Bootstrap было выявлен регион с наилучшими показателями – geo-1.   Доверительный интервал у данной модели находится в пределах от 33 млн. до 852 млн., а средняя выручка составляет 456 млн. Так же у данной модели наименьший риск убытков - 1,5%. 